# Step 0. Install LAMA

In [ ]:
pip install lightautoml

# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import os
import time

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.whitebox_presets import WhiteBoxPreset
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDecoWhitebox

# Step 0.2. Parameters 

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

# Step 0.3. Fix torch number of threads and numpy seed 

In [ ]:
np.random.seed(RANDOM_STATE)

# Step 0.4. Example data load 

In [ ]:
%%time

data = pd.read_csv('../input/lama-datasets/sampled_app_train.csv')
data.head()

# Step 0.5. (Optional) Some user feature preparation 

Cell below shows some user feature preparations to create task more difficult (this block can be omitted if you don't want to change the initial data):

In [ ]:
%%time

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data['report_dt'] = np.datetime64('2018-01-01')

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

# Step 0.6. (Optional) Data splitting for train-test 

Block below can be omitted if you are going to train model only or you have specific train and test files:

In [ ]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

In [ ]:
train_data.head()

#  ==== AutoML preset usage ====


## Step 1. Create Task

In [ ]:
%%time

task = Task('binary', )

## Step 2. Setup columns roles

Roles setup here set target column and base date, which is used to calculate date differences:

In [ ]:
%%time

roles = {'target': TARGET_NAME,
         DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt',
         }

## Step 3. Create AutoML from preset

To create AutoML model here we use `WhiteBoxPreset` preset.


All params we set above can be send inside preset to change its configuration:

In [ ]:
%%time 

automl = WhiteBoxPreset(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params={'report': True})

RD = ReportDecoWhitebox()
automl_rd = RD(automl)

oof_pred = automl_rd.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Save new config template:

In [ ]:
automl.get_config(path='wb_config.yml')

## Step 4. Predict to test data and check scores

In [ ]:
%%time

test_pred = automl_rd.predict(test_data, report=True)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred, test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
print('TEST score: {}'.format(roc_auc_score(test_data[TARGET_NAME].values,
                                            test_pred.data[:, 0])))

## Step 5. Report 

Report is [here](./lama_report/lama_interactive_report.html)